Ref: https://github.com/SchindlerLiang/VAE-for-Anomaly-Detection

In [1]:
import os
import random

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from evaluator import evaluate
from data_loader import load_kdd_cup_urc, load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4, load_power_demand # Univariate Datasets
from data_loader import load_nasa, load_ecg, load_gesture, load_smd # Multivariate Datasets

from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm

# THESE LINES ARE FOR REPRODUCIBILITY
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [4]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=1
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [5]:
def LSTM_VAE(X_train):
    latent_dim = 16

    encoder_inputs = keras.Input(shape=(X_train.shape[1], X_train.shape[2]))
    x = layers.LSTM(64, return_sequences=True)(encoder_inputs)
    x = layers.LSTM(32)(x)
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.RepeatVector(X_train.shape[1])(latent_inputs)
    x = layers.LSTM(32, return_sequences=True)(x)
    x = layers.LSTM(64)(x)
    x = layers.Dense(X_train.shape[1] *  X_train.shape[2])(x)
    decoder_outputs = layers.Reshape([X_train.shape[1], X_train.shape[2]])(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

    model = VAE(encoder, decoder)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
    history = model.fit(X_train, epochs=50, batch_size=128, verbose=0)
    return model

### Yahoo S5

In [6]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [7]:
for loader in [load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4]:
    datasets = loader(128, 64)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
    
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

yahoo_A1 0.5714285142857172 0.1250000006249989 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.33333329444444565 0.09999998800000123 0.49999993750000654
yahoo_A1 0.6666666000000033 0.24999998333333406 0.33333330555555735
yahoo_A1 0.666666577777783 0.37499995687500437 0.4999999583333361
yahoo_A1 0.33333329444444565 0.09999998800000123 0.3749999531250049
yahoo_A1 0.8888888197530895 0.4166666460416676 -0.0
yahoo_A1 0.5714285142857172 0.1250000006249989 -0.0
yahoo_A1 0.33333329444444565 0.09999998800000123 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.33333329444444565 0.09999998800000123 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.7999999200000041 0.5416666119444498 0.7499999250000059
yahoo_A1 0.6666666000000033 0.16666666527777654 -0.0
yahoo_A1 0.6666666000000033 0.16666666527777654 -0.0
yahoo_A1 0.7499999343750033 0.1666666744444422 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.7499999343750033 0.2916666525694

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/92 [00:00<?, ?it/s]

yahoo_A3 0.33333329444444565 0.09999998800000123 0.0
yahoo_A3 0.33333329444444565 0.16249997831250235 0.12499999062500056
yahoo_A3 0.7499999343750033 0.16666667444444216 -0.0
yahoo_A3 0.4999999500000025 0.10000000133333238 -0.0
yahoo_A3 0.8888888197530895 0.25000000993055255 -0.0
yahoo_A3 0.33333329444444565 0.14999997950000227 -0.0
yahoo_A3 0.7999999360000031 0.24999999777777704 -0.0
yahoo_A3 0.5714285142857172 0.1250000006249989 -0.0
yahoo_A3 0.7999999360000031 0.16666667972221952 -0.0
yahoo_A3 0.999999930000003 0.4999999800000005 0.0
yahoo_A3 0.8888888197530895 0.25000000993055255 -0.0
yahoo_A3 0.9090908429752097 0.2500000140138856 -0.0
yahoo_A3 0.28571425306122533 0.08333332361111209 -0.0
yahoo_A3 0.7999999360000031 0.24999999777777704 -0.0
yahoo_A3 0.28571425306122533 0.08333332361111209 -0.0
yahoo_A3 0.8888888197530895 0.25000000993055255 -0.0
yahoo_A3 0.33333329444444565 0.09999998800000123 -0.0
yahoo_A3 0.4999999500000025 0.10000000133333238 -0.0
yahoo_A3 0.5714285142857172 0.1

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

yahoo_A4 0.6666666074074105 0.1250000081249981 -0.0
yahoo_A4 0.33333329444444565 0.09999998800000123 -0.0
yahoo_A4 0.6666666074074105 0.18749999718749943 -0.0
yahoo_A4 0.33333329444444565 0.09999998800000123 -0.0
yahoo_A4 0.8571427836734729 0.2500000034722198 -0.0
yahoo_A4 0.7499999343750033 0.24999999249999974 0.4999999250000088
yahoo_A4 0.4999999500000025 0.29166663694444717 0.6249999281250072
yahoo_A4 0.4999999500000025 0.10000000133333238 -0.0
yahoo_A4 0.9090908429752097 0.37499998901388903 -0.0
yahoo_A4 0.7999999360000031 0.16666667972221952 -0.0
yahoo_A4 0.9090908429752097 0.2500000140138856 -0.0
yahoo_A4 0.6666666074074105 0.1250000081249981 -0.0
yahoo_A4 0.5714285142857172 0.1250000006249989 -0.0
yahoo_A4 0.28571425306122533 0.08333332361111209 -0.0
yahoo_A4 0.28571425306122533 0.08333332361111209 -0.0
yahoo_A4 0.4999999500000025 0.10000000133333238 -0.0
yahoo_A4 0.8888888197530895 0.25000000993055255 -0.0
yahoo_A4 0.7499999343750033 0.1666666744444422 -0.0
yahoo_A4 0.999999930

In [8]:
yahoo_results = pd.DataFrame(total_scores)
yahoo_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
yahoo_A1,0.422151,0.141964,0.098214
yahoo_A2,0.361276,0.075546,0.082991
yahoo_A3,0.655396,0.194173,0.018569
yahoo_A4,0.653887,0.192446,0.055894


### NASA

In [9]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [10]:
for loader in [load_nasa]:
    datasets = loader(128, 64)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
        
        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

D1 0.25772800179650046 0.1399403712646333 0.49129400847906457
D2 0.2848620187799369 0.12504215926087756 0.46294618259259673


In [11]:
nasa_results = pd.DataFrame(total_scores)
nasa_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.257728,0.139940,0.491294
D2,0.284862,0.125042,0.462946


### SMD

In [12]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [13]:
for loader in [load_smd]:
    datasets = loader(128, 64)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

smd 0.23153690558205123 0.06546275382269463 0.49999999900806086
smd 0.14105792132682896 0.0 0.0
smd 0.18546364222837944 0.06328834236421746 0.5153655145434519
smd 0.17412933725031698 0.06972508131401133 0.49051896042181853
smd 0.08333332530382016 0.02173913029300567 0.5014164273190038
smd 0.4793388063315375 0.23169968983952827 0.38553208966378616
smd 0.29561198390945814 0.1179091731888085 0.43671373799419855
smd 0.23728811456478205 0.07913685087716121 0.38249760046817965
smd 0.21553882777872163 0.08900926340511002 0.47055018015055705
smd 0.30092590022183857 0.08855585815025727 0.5032894727443633
smd 0.12690354134994566 0.02975852977456804 0.5400136767430757
smd 0.29953914490114675 0.13963483542105232 0.4377402688730397
smd 0.2641509203453206 0.09249582775395099 0.369470041035989
smd 0.08583690161911327 0.029305960383263503 0.4572675824499064
smd 0.21411190291793372 0.05557691507824329 0.3780248895960818
smd 0.026737965262089488 0.006775067613340092 0.4999999898626375
smd 0.190283383514

In [14]:
smd_results = pd.DataFrame(total_scores)
smd_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
smd,0.180453,0.052692,0.398041


### ECG

In [15]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [16]:
for loader in [load_ecg]:
    datasets = loader(64, 32)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

D1 0.3174602897455299 0.14846535819132492 0.46366027628276396
D2 0.3043477989603044 0.10399022707154182 0.2723214231827966
D3 0.24999996493055968 0.10674128438902575 0.6841924263725396
D4 0.3243242953981028 0.13406207910861692 0.5374999865714288
D5 0.27450977916186264 0.1414382336174631 0.4498069408655706
D6 0.20895520454444344 0.076324127178433 0.39976957885927006
D7 0.09999997655556071 0.031852760742268144 0.5921324962186367
D8 0.2622950559527031 0.10932404123601588 0.5412912848082317
D9 0.44444440138271973 0.3200539607397437 0.5621301718057091


In [17]:
ecg_results = pd.DataFrame(total_scores)
ecg_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.317460,0.148465,0.463660
D2,0.304348,0.103990,0.272321
D3,0.250000,0.106741,0.684192
D4,0.324324,0.134062,0.537500
D5,0.274510,0.141438,0.449807
D6,0.208955,0.076324,0.399770
D7,0.100000,0.031853,0.592132
D8,0.262295,0.109324,0.541291
D9,0.444444,0.320054,0.562130


### Power Demand

In [18]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [19]:
for loader in [load_power_demand]:
    datasets = loader(512, 256)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc']) 

  0%|          | 0/1 [00:00<?, ?it/s]

power_demand 0.7499999343750033 0.16666667444444216 0.2499999625000044


In [20]:
power_results = pd.DataFrame(total_scores)
power_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
power_demand,0.75,0.166667,0.25


### 2D Gesture

In [21]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [22]:
for loader in [load_gesture]:
    datasets = loader(64, 32)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]

        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/1 [00:00<?, ?it/s]

gesture 0.44230765700813873 0.41932892728918425 0.594893026124482


In [23]:
gesture_results = pd.DataFrame(total_scores)
gesture_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
gesture,0.442308,0.419329,0.594893
